# Initial data cleaning

In this notebook we will clean and prepare the data for the analysis. We will perform the following steps:

1. Load the data
2. Eliminate irrelevant columns
3. Check for missing values
4. Handle missing values
5. Check unique values per column and filter categorical data
6. Save the clean data

In [1]:
# Importing libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

sns.set_style("darkgrid")
sns.set_palette("Set1")

c:\Users\juanj\OneDrive - UPB\Estructurados\Final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data and eliminate irrelevant columns

df_train = pd.read_csv("../Data/Raw/train.csv")
df_test = pd.read_csv("../Data/Raw/test.csv")
df_train = df_train.drop(columns="id")
df_test = df_test.drop(columns="id")

In [3]:
# Data type correction

types = {column: 'category' for column in df_train.columns if df_train[column].dtype == 'object'}
df_train = df_train.astype(types)
types.pop("class")
df_test = df_test.astype(types)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116945 entries, 0 to 3116944
Data columns (total 21 columns):
 #   Column                Dtype   
---  ------                -----   
 0   class                 category
 1   cap-diameter          float64 
 2   cap-shape             category
 3   cap-surface           category
 4   cap-color             category
 5   does-bruise-or-bleed  category
 6   gill-attachment       category
 7   gill-spacing          category
 8   gill-color            category
 9   stem-height           float64 
 10  stem-width            float64 
 11  stem-root             category
 12  stem-surface          category
 13  stem-color            category
 14  veil-type             category
 15  veil-color            category
 16  has-ring              category
 17  ring-type             category
 18  spore-print-color     category
 19  habitat               category
 20  season                category
dtypes: category(18), float64(3)
memory usage: 124.9 MB


In [4]:
# Checking missing values per column

missing = df_train.isna().mean() * 100
missing

class                    0.000000
cap-diameter             0.000128
cap-shape                0.001283
cap-surface             21.528227
cap-color                0.000385
does-bruise-or-bleed     0.000257
gill-attachment         16.809280
gill-spacing            40.373988
gill-color               0.001829
stem-height              0.000000
stem-width               0.000000
stem-root               88.452732
stem-surface            63.551362
stem-color               0.001219
veil-type               94.884350
veil-color              87.936970
has-ring                 0.000770
ring-type                4.134818
spore-print-color       91.425482
habitat                  0.001444
season                   0.000000
dtype: float64

### Missing values conclussion:

As seen in the missing values analysis, there are columns missing a lot of values. We will drop any column with more than 30% missing values. We will then drop all the rows with missing values as there is a lot of data and we can afford to lose some of it.

In [5]:
high_missing = df_train.columns[missing > 30]

df_train = df_train.drop(columns=high_missing)
df_test = df_test.drop(columns=high_missing)

df_train = df_train.dropna()
df_test = df_test.dropna()

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1930108 entries, 0 to 3116944
Data columns (total 15 columns):
 #   Column                Dtype   
---  ------                -----   
 0   class                 category
 1   cap-diameter          float64 
 2   cap-shape             category
 3   cap-surface           category
 4   cap-color             category
 5   does-bruise-or-bleed  category
 6   gill-attachment       category
 7   gill-color            category
 8   stem-height           float64 
 9   stem-width            float64 
 10  stem-color            category
 11  has-ring              category
 12  ring-type             category
 13  habitat               category
 14  season                category
dtypes: category(12), float64(3)
memory usage: 81.0 MB


### Checking unique values per column

In [7]:
categorical = df_train.select_dtypes(include="category").columns.drop("class")

for column in categorical:
    unique = df_train[column].nunique()
    print(f"{column}: {unique}")

cap-shape: 48
cap-surface: 76
cap-color: 52
does-bruise-or-bleed: 21
gill-attachment: 64
gill-color: 49
stem-color: 50
has-ring: 18
ring-type: 37
habitat: 43
season: 4


In [8]:
for column in categorical:
    print(f"\nRows per category in: \n{df_train[column].value_counts().head(10)}")


Rows per category in: 
cap-shape
x    983649
f    450586
s    239475
b    127770
p     56234
c     55885
o     16166
d        48
e        34
n        28
Name: count, dtype: int64

Rows per category in: 
cap-surface
t    406544
s    343735
h    217348
g    203429
d    197858
y    185567
e     87558
w     85703
k     85415
i     71639
Name: count, dtype: int64

Rows per category in: 
cap-color
n    866525
w    240022
y    207793
e    137490
o    134369
g     80768
p     78930
k     45505
r     41267
u     40346
Name: count, dtype: int64

Rows per category in: 
does-bruise-or-bleed
f    1529827
t     400216
h          9
w          9
c          7
a          6
y          4
b          4
d          4
p          3
Name: count, dtype: int64

Rows per category in: 
gill-attachment
a    512171
d    418865
x    258594
s    241767
e    238960
p    226445
f     32915
c        55
u        42
t        32
Name: count, dtype: int64

Rows per category in: 
gill-color
w    586808
n    364863
y    336556


### Unique values conclussion:

Here it is clear that there are columns with a lot of unique values and most of that unique values only appear a couple of times. To reduce the amount of unique values we will unify all categories with less than 70 appearances into a new category called 'Other'.

In [9]:
def unify_categories(df, column, threshold=70):
    counts = df[column].value_counts()
    infrequent = counts[counts <= threshold].index
    df[column] = df[column].apply(lambda x: "Other" if x in infrequent else x)
    return df

for column in categorical:
    df_train = unify_categories(df_train, column)
    df_test = unify_categories(df_test, column)

In [10]:
# After unifying categories

for column in categorical:
    unique = df_train[column].nunique()
    print(f"{column}: {unique}")

cap-shape: 8
cap-surface: 13
cap-color: 13
does-bruise-or-bleed: 3
gill-attachment: 8
gill-color: 13
stem-color: 13
has-ring: 3
ring-type: 8
habitat: 9
season: 4


In [11]:
# Duplicate removal

df_train = df_train.drop_duplicates()
df_test = df_test.drop_duplicates()

In [12]:
df_train.to_csv("../Data/PartiallyClean/train.csv", index=False)
df_test.to_csv("../Data/PartiallyClean/test.csv", index=False)